# Statistical Thinking in Python (Part 2)

## 1 Parameter estimation by optimization

### 1.1 Optimal parameters  

* 如果你假设数据服从某个分布(例如normal distribution)，而数据真的是这个分布
    * calculate parameter from sample data and plug-it-in np.random.normal 
    * CDF of theoretical population data overlays with ECDF of sample data, then yes you got the optimal params
* 如果你假设数据服从某个分布(例如normal distribution)，而数据**并不是**这个分布
    * model is wrong, then your np.mean np.std makes no sense
    * specifically, when doing linear regression, rely on built-in func to find optimal params.

### 1.2 Linear regression by least squares
* It is always a good idea to do some EDA ahead of our analysis
    * visual EDA, eg.scatter plot
    * quantitative EDA, eg.percentile, pearson correlation coefficient
* If result of EDA allows, perform a linear regression

In [4]:
# this module contains all func I defined for this course
from stats_thinking import *

In [5]:
pearson_r

<function stats_thinking.pearson_r>

In [ ]:
# Plot the illiteracy rate versus fertility
_ = plt.plot(illiteracy, fertility, marker='.', linestyle='none')
plt.margins(0.02)
_ = plt.xlabel('percent illiterate')
_ = plt.ylabel('fertility')

# Perform a linear regression using np.polyfit(): a, b
a, b = np.polyfit(illiteracy, fertility, 1)

# Print the results to the screen
print('slope =', a, 'children per woman / percent illiterate')
print('intercept =', b, 'children per woman')

# Make theoretical line to plot
x = np.array([0, 100])
y = a * x + b

# Add regression line to your plot
_ = plt.plot(x, y)

# Draw the plot
plt.show()


### 1.3 The importance of EDA: Anscombe's quartet
* 下面这四个图 x-mean, y-mean, 直线拟合后的slope和intercept，sum of squared residual几乎完全相等
* 然而并不是每个都最适合用直线来拟合
* so look before you leap, EDA before any other analysis

![](http://upload-images.jianshu.io/upload_images/1526845-242eafdc39207d35.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

## 2 Bootstrap confidence intervals 

### 2.1 Generating bootstrap replicates
* bootstrap -->> resample with replacement
* bootstrap sampele -->> a resampled array of data
* bootstrap replicate -->> a summary statistic computed from a bootstrap sample

In [ ]:
np.random.choice(data, len(data))  # draw one bs sample with the same length 
                                   # as source data

In [ ]:
# Visualizing bootstrap samples

    # 50 bs sample and plot its ecdf
for i in range(50):
    bs_sample = np.random.choice(rainfall, size=len(rainfall))

    x, y = ecdf(bs_sample)
    _ = plt.plot(x, y, marker='.', linestyle='none',
                 color='gray', alpha=0.1)

    # Compute and plot ECDF from original data
x, y = ecdf(rainfall)
_ = plt.plot(x, y, marker='.')

    # Make margins and label axes
plt.margins(0.02)
_ = plt.xlabel('yearly rainfall (mm)')
_ = plt.ylabel('ECDF')

    # Show the plot
plt.show()


### 2.2 Bootstrap confidence intervals
* 侧光速的那个家伙并不care这具体的100个sample如何如何；他想知道的是光速population到底咋分布
* 用手头100个sample, resample with replacement，重复N次，就得到了N个size=100的sample；相当于从population里重新draw了N次。
* N个sample有N个均值 -->>> get a sampling distribution --->> compute confidence interval using np.percentile
    * confidence interval: If we repeated measurements over and over again, p% of the observed values would lie within the p% confidence interval.

In [ ]:
# I've copied this func in 'stats_thinking' module

def bootstrap_replicate_1d(data, func):
    '''
    Take in an array and a aggregate function.
    Return one boostrap statistic by passing bs sample to the func.
    '''
    return func(np.random.choice(data, size=len(data)))

In [ ]:
# I've copied this func in 'stats_thinking' module

def draw_bs_reps(data, func, size=1):
    """
    Draw bootstrap replicates.
    Return an array of statistics computed from bs samples.
    """
    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates

### 2.3 Pairs bootstrap
* 之前的confidence interval就手头的1D array操作，并不假设它们符合什么模型
* 对于假设它们符合linear model的两个array而言，需要pair-wise resample
    * resample from np.arange([0, len(x)])  as indices
    * use indices to index x-y pairs
    * do linear regression using np.polyfit(x_bs, y_bs, 1)multiple times, get multiple statistic(slope & intercept)
    * get confidence interval using np.percentile
    * or can plot multiple regression lines

## 3 Introduction to hypothesis testing 

### 3.1 Formulating and simulating a hypothesis
* permutation, used to test if two variables have identical probability distributions. Operation:
    * 把两个array合并作一个，打乱重新排序
    * 假装前n1个是variable 1，后n2个是variable 2。n1 n2是var1 var2长度
    * calculate test stat and p-value

In [ ]:
# Generating permutation sample
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

### 3.2 Test statistics and p-values
* 如何来衡量whether or not null hypothesis is true --->> choose a test statistic
    * test statistic
        * A single number that can be computed from observed data and from data you simulate under the null hypothesis
        * It serves as a basis of comparison between the two.
        * when choosing a test statistic, The most important thing to consider is: What are you asking?
* how can a test statistic indicate whether the null hypothesis is true  --->>> p-value
    * p-value
        * The probability of obtaining a value of your test statistic that is **at least as extreme as **what was observed, under the assumption the null hypothesis is true

a permutation replicate is a single value of a statistic computed from a permutation sample

In [ ]:
# Generating permutation replicates

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

In [ ]:
# when using a permutation test with a test statistic 
# of the difference of means to test null hypothesis.

    # this is the func
def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1) - np.mean(data_2)

    return diff

In [ ]:
# an example

    # empirical test stat from observed data
empirical_diff_means = diff_of_means(force_a, force_b)

    # 10000 theoretical test stat from generated data
perm_replicates = draw_perm_reps(force_a, force_b,
                                 diff_of_means, size=10000)

    # compare empirical test stat to theoretical test stat: compute p-value: p
p = np.sum(perm_replicates >= empirical_diff_means) / len(perm_replicates)

    # Print the result
print('p-value =', p)

### 3.3 Bootstrap hypothesis tests

![](http://upload-images.jianshu.io/upload_images/1526845-61864c942922e0b6.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

* one sample test
    * campare one set of data into a single number
    * 人话解读之课程案例：  
    课程中的例子是比较老头A测的a set of 光速和老头B测的光速（只有一个值）    
    H_null: 会不会老头B的单一的值才是真正的均值呢？老头A的data set只是个偶然。  
    --> 假设H_null成立，老头A的data set是偶然，那就不能用老头A的数据去模拟生成N次实验；  
    --> 可以用老头B的单一值作为均值 + 老头A的数据的variance去模拟从population中取N次sample（这里暗含的假设是老头A的variance是有代表性的，老头A只是均值没有代表性）  
    --> shifted arrA: arrA - np.mean(arrA) + arrB   
    --> emprical test stat: np.mena(arrA) - arrB  
    --> bootstrap sample and bootstrap replicate based one shifted arrA(相当于重做N次实验) --> N个test stat
    --> compare empirical test stat and experimental test stat, p-value
    
    
* two sample test
    * compare two sets of data: concatenate -> bs_resample -> each sample gives a bootstrap replicate(test statistic) -> compare test stat from observed data with them -> get p-value
    * The permutation test exactly simulates the null hypothesis that the data come from the same distribution, whereas the bootstrap test approximately simulates it. As we will see, though, the bootstrap hypothesis test, while approximate, is more versatile.

#### compare permutation and bootstrap
* permutation
    *  two-sample hypothesis test ✔️/ one-sample hypothesis test ❌
    * whether two sample have the same distribution✔️/ just have the same mean, not necessarily same dist. ❌
    
* bootstrap 
    * two-sample hypothesis test ✔️/ one-sample hypothesis test ✔️
    * whether two sample have the same distribution ✔️ / just have the same mean, not necessarily same dist ✔️

## 4 Hypothesis test examples

### 4.1 A/B testing

The vote for the Civil Rights Act in 1964
100xp
The Civil Rights Act of 1964 was one of the most important pieces of legislation ever passed in the USA. Excluding "present" and "abstain" votes, 153 House Democrats and 136 Republicans voted yay. However, 91 Democrats and 35 Republicans voted nay. Did party affiliation make a difference in the vote?

To answer this question, you will evaluate the hypothesis that the party of a House member has no bearing on his or her vote. You will use the fraction of Democrats voting in favor as your test statistic and evaluate the probability of observing a fraction of Democrats voting in favor at least as small as the observed fraction of 153/244. (That's right, at least as small as. In 1964, it was the Democrats who were less progressive on civil rights issues.) To do this, permute the party labels of the House voters and then arbitrarily divide them into "Democrats" and "Republicans" and compute the fraction of Democrats voting yay.

In [ ]:
# Construct arrays of data: dems, reps
dems = np.array([True] * 153 + [False] * 91)
reps = np.array([True] * 136 + [False] * 35)

def frac_yay_dems(dems, reps):
    """Compute fraction of Democrat yay votes."""
    frac = np.sum(dems) / len(dems)
    return frac

# Acquire permutation samples: perm_replicates
perm_replicates = draw_perm_reps(dems, reps, frac_yay_dems, 10000)

# Compute and print p-value: p
p = np.sum(perm_replicates <= 153/244) / len(perm_replicates)
print('p-value =', p)


### 4.2 Test of correlation
（突然意识到用hacker stats来搬砖一个巨大的好处是很灵活，不用背鬼扯的公式。。。。）

The observed correlation between female illiteracy and fertility in the data set of 162 countries may just be by chance; the fertility of a given country may actually be totally independent of its illiteracy. You will test this null hypothesis in the next exercise.

To do the test, you need to simulate the data assuming the null hypothesis is true. Of the following choices, which is the best way to to do it?

* A.Do a bootstrap sampling in which you sample 162 illiteracy values with replacement and then 162 fertility values with replacement.
* ✔️B.Do a permutation test: Permute the illiteracy values but leave the fertility values fixed to generate a new set of (illiteracy, fertility) data.
* C.Do a permutation test: Permute both the illiteracy and fertility values to generate a new set of (illiteracy, fertility data).  

选项B比A更严谨，比C更computationally cheap

The observed correlation between female illiteracy and fertility may just be by chance; the fertility of a given country may actually be totally independent of its illiteracy. You will test this hypothesis. To do so, permute the illiteracy values but leave the fertility values fixed. This simulates the hypothesis that they are totally independent of each other. For each permutation, compute the Pearson correlation coefficient and assess how many of your permutation replicates have a Pearson correlation coefficient greater than the observed one.

The function pearson_r() that you wrote in the prequel to this course for computing the Pearson correlation coefficient is already in your name space.

In [ ]:
# Compute observed correlation: r_obs
r_obs = pearson_r(illiteracy, fertility)

# Initialize permutation replicates: perm_replicates
perm_replicates = np.empty(10000)

# Draw replicates
for i in range(10000):
    # Permute illiteracy measurments: illiteracy_permuted
    illiteracy_permuted = np.random.permutation(illiteracy)

    # Compute Pearson correlation
    perm_replicates[i] = pearson_r(illiteracy_permuted, fertility)

# Compute p-value: p
p = np.sum(perm_replicates >= r_obs) / len(perm_replicates)
print('p-val =', p)

## 5 Putting it all together: a case study 